In [14]:
import numpy as np
import pandas as pd 
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [15]:
data = pd.read_csv(r"Practical5.csv")
X = data.iloc[:, [1, 2, 3, 4, 5, 6, 7]].values
y = data.iloc[:, -1].values

In [16]:
data.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [17]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:,0] = le.fit_transform(X[:,0])

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [19]:
dt_classifier = DecisionTreeClassifier()


In [20]:
param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [3, 5, 7, 10],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}


In [21]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)

In [22]:
# Create a GridSearchCV object
grid_search = GridSearchCV(dt_classifier, param_grid, cv=kf, scoring='accuracy')

# Fit the GridSearchCV object to the training data
grid_search.fit(X_train, y_train)

# Get the best parameters and best score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

# Instantiate the Decision Tree Classifier with the best parameters
best_dt_classifier = DecisionTreeClassifier(**best_params)

# Train the classifier with the entire training data
best_dt_classifier.fit(X_train, y_train)

# Make predictions on the test data
y_pred = best_dt_classifier.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print("Best Parameters:", best_params)
print("Best Score:", best_score)

Accuracy: 0.7662337662337663
Best Parameters: {'criterion': 'entropy', 'max_depth': 3, 'min_samples_leaf': 4, 'min_samples_split': 2}
Best Score: 0.7442756230840997


In [23]:
from sklearn.tree import export_text

# Instantiate the Decision Tree Classifier with the best parameters
best_dt_classifier = DecisionTreeClassifier(**best_params)

# Train the classifier with the entire training data
best_dt_classifier.fit(X_train, y_train)

# Get column names from the DataFrame
feature_names = data.columns[[1, 2, 3, 4, 5, 6, 7]].tolist()

# Export the decision tree as text
tree_rules = export_text(best_dt_classifier, feature_names=feature_names)
print(tree_rules)



|--- Glucose <= 65.50
|   |--- Age <= 28.50
|   |   |--- BMI <= 30.95
|   |   |   |--- class: 0
|   |   |--- BMI >  30.95
|   |   |   |--- class: 0
|   |--- Age >  28.50
|   |   |--- BMI <= 26.35
|   |   |   |--- class: 0
|   |   |--- BMI >  26.35
|   |   |   |--- class: 0
|--- Glucose >  65.50
|   |--- BMI <= 29.95
|   |   |--- Glucose <= 83.50
|   |   |   |--- class: 0
|   |   |--- Glucose >  83.50
|   |   |   |--- class: 1
|   |--- BMI >  29.95
|   |   |--- Glucose <= 103.50
|   |   |   |--- class: 1
|   |   |--- Glucose >  103.50
|   |   |   |--- class: 1



In [24]:
from sklearn.tree import plot_tree
import matplotlib.pyplot as plt
 
class_names = data.columns[[8]].tolist()
# Plot the decision tree
plt.figure(figsize=(100,100))
plot_tree(best_dt_classifier, feature_names=feature_names, filled=True)
plt.show()
